In [1]:
# Load necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import glob
from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error
from skopt import gp_minimize
import tensorflow as tf
import tensorflow_probability as tfp


2024-05-18 19:18:44.409857: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-18 19:18:44.537793: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-18 19:18:45.123278: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-18 19:18:47.387565: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
## Load and sort data

# Set the directory where your data files are located
data_dir = '/home/jwm/data/JASCO_FP_6500_Spectrofluorometer/2024april13_50mM_DAB2_418_630_150mM_NaCl_1DCVJ_slow_hysteresis/'

# Get all txt files in the directory
data_files = glob.glob(data_dir + "*.txt")

# Initialize a list to store loaded data
loaded_data = []

# Initialize variables for baseline data
baseline_data = None
baseline_title = None

# Function to load (x, y) data from txt file starting from the 19th line
def load_data(file_path):
    return np.loadtxt(file_path, skiprows=18)

# Load data from each file
for file_path in data_files:
    loaded_data.append(load_data(file_path))

titles = []

# Extract 6th to 4th last digits from file name and use as title
titles = [int(file_path.split('/')[-1].split('.')[0][-7:-4]) for file_path in data_files]

# Sort the titles and data_files in ascending order
sorted_indices = np.argsort(titles)
titles = [titles[i] for i in sorted_indices]
data_files = [data_files[i] for i in sorted_indices]
loaded_data = [loaded_data[i] for i in sorted_indices]

# Sort the files by their title
data_files.sort(key=lambda x: int(x.split('/')[-1].split('.')[0][-7:-4]))
    
# Check if file name contains "999" and assign it as baseline data
if "999" in file_path:
    baseline_data = loaded_data[-1]
    baseline_title = title
    
print(titles)

# Extract x-values from the first spectrum in loaded_data
wavelengths = loaded_data[0][2:, 0].astype(float)


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 999]


In [ ]:
## Set Parameters

# Set the number of the first and last spectrum to use in the plot -1
first_spectrum = 0
last_spectrum = 30

# Define the x-values to extract (takes average of 10 data points)
x_values = [465, 535]

# Define the range of titles you want to use for the biexponential fit
start_title = 8
end_title = 30



## Extract respective intensities for chosen spectra and wavelenghts, averaged over 1nm

# Initialize lists to store y-values for each x-value for all spectra
y_values_all = [[] for _ in range(len(x_values))]

# Initialize lists to store x-values for each x-value for all spectra
x_values_all = [[] for _ in range(len(x_values))]

# Iterate through spectra between the first and last
for i, data in enumerate(loaded_data[first_spectrum:last_spectrum+1]):
    # Iterate through each specified x-value
    spectrum_index = first_spectrum + i
    spectrum_title = titles[spectrum_index]  # Assuming titles list is correctly populated
#   print(f"Spectrum Index: {spectrum_index}, Spectrum Title: {spectrum_title}")
    for j, x in enumerate(x_values):
        # Find the indices of the 10 closest x-values to the current x
        closest_indices = np.argsort(np.abs(data[:, 0] - x))[:10]
        # Extract the corresponding x-values
        closest_x_values = data[closest_indices, 0]
#       print(f"Closest x-values for x={x}: {closest_x_values}")
# Calculate the average x-value
        avg_x_value = np.mean(closest_x_values)
        x_values_all[j].append(avg_x_value)
        # Extract the corresponding y-values
        y_values = data[closest_indices, 1]
#       print(f"Closest y-values for x={x}: {y_values}")
        # Calculate the average y-value
        avg_y_value = np.mean(y_values)
#       print(f"Average y-value for x={x}: {avg_y_value}")
        # Store the average y-value
        y_values_all[j].append(avg_y_value)
print(f"Average y-values:", y_values_all)

#print(closest_indices)
closest_x_values = data[closest_indices, 0]
#print(f"Closest x-values for x={x}: {closest_x_values}")

## Fit Polynomial

# Generate a continuous range of x-values covering all data points for evaluation of fitted functions
x_values_interp = np.linspace(0, len(y_values_all[0]) - 1)

print(x_values_interp)

# Initialize the array to store the interpolated y-values for all x-values
y_values_interp_all = []

# Define the biexponential function
def biexponential(x, a1, b1, c1, a2, b2, c2):
    return a1 * np.exp(-b1 * x) + c1 + a2 * np.exp(-b2 * x) + c2
    
# Initialize the array to store the interpolated y-values for all x-values
y_values_biexp_interp_all = []

# Initialize equation display box
equation_box = ""




## Fit Biexponential function to the data

# Convert extracted digits back to temperature values
# temperatures = [10 + i * 0.5 for i in range(len(titles))]


# Convert extracted digits back to temperature values for the selected range of titles
temperatures = [10 + i * 0.5 for i, title in enumerate(titles) if first_spectrum <= i <= last_spectrum]


print("List of temperatures", temperatures)


# Convert titles to numerical format if necessary
numeric_titles = [float(title) for title in titles]

# Initialize filtered temperatures list
filtered_temperatures_all = []

# Filter temperatures based on title range
for i, title in enumerate(numeric_titles):
    if start_title <= title <= end_title:
        filtered_temperatures_all.append(temperatures[i])

# Print filtered_temperatures_all
print(filtered_temperatures_all)

# Initialize filtered y-values list
filtered_y_values_all = [[] for _ in range(len(x_values))]

# Filter y_values_all based on title range
for i, title in enumerate(numeric_titles):
    if start_title <= title <= end_title:
        for j in range(len(x_values)):
            filtered_y_values_all[j].append(y_values_all[j][i])

# Print filtered_y_values_all
print(filtered_y_values_all)

# Round all y-values to two decimals
filtered_y_values_all = np.round(filtered_y_values_all, decimals=2)
print(filtered_y_values_all)


# Iterate over each set of y-values (corresponding to different x-values)
for y_values in y_values_all:
    # Construct polynomial function and get coefficients
    coeffs = np.polyfit(temperatures, y_values, 5)
    poly = np.poly1d(coeffs)
    
    # Evaluate the polynomial at the continuous x-values
    y_values_interp = poly(temperatures)
    
    # Store the interpolated y-values for this x-value
    y_values_interp_all.append(y_values_interp)

# Construct equation string
    equation = f'Polynomial Fit: {x} nm: y = {coeffs[0]:.2f}x^5 + {coeffs[1]:.2f}x^4 + {coeffs[2]:.2f}x^3 + {coeffs[3]:.2f}x^2 + {coeffs[4]:.2f}x + {coeffs[5]:.2f}'
    equation_box += equation
    equation_box += '\n'

print(y_values_interp_all)



import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np

# Define the biexponential function
def biexponential(x, a1, b1, c1, a2, b2, c2):
    return a1 * tf.exp(-b1 * x) + c1 + a2 * tf.exp(-b2 * x) + c2

# Function to perform MCMC sampling for fitting
def perform_biexponential_fit(x_data, y_data, num_results=1000, num_burnin_steps=500):
    # Define the model
    def model():
        a1 = yield tfp.distributions.Normal(loc=1.0, scale=10.0, name='a1')
        b1 = yield tfp.distributions.Normal(loc=1.0, scale=10.0, name='b1')
        c1 = yield tfp.distributions.Normal(loc=1.0, scale=10.0, name='c1')
        a2 = yield tfp.distributions.Normal(loc=1.0, scale=10.0, name='a2')
        b2 = yield tfp.distributions.Normal(loc=1.0, scale=10.0, name='b2')
        c2 = yield tfp.distributions.Normal(loc=1.0, scale=10.0, name='c2')
        y = yield tfp.distributions.Normal(
            loc=biexponential(x_data, a1, b1, c1, a2, b2, c2),
            scale=1.0,
            name='y'
        )

    # Define the model and sample from it
    model = tfp.distributions.JointDistributionCoroutineAutoBatched(model)
    target_log_prob_fn = lambda a1, b1, c1, a2, b2, c2: model.log_prob((a1, b1, c1, a2, b2, c2, y_data))

    # Initialize the HMC transition kernel
    kernel = tfp.mcmc.HamiltonianMonteCarlo(
        target_log_prob_fn=target_log_prob_fn,
        num_leapfrog_steps=3,
        step_size=0.1
    )

    # Define the initial state
    initial_state = [
        tf.constant(1.0),
        tf.constant(1.0),
        tf.constant(1.0),
        tf.constant(1.0),
        tf.constant(1.0),
        tf.constant(1.0)
    ]

    # Sample using MCMC
    samples, kernel_results = tfp.mcmc.sample_chain(
        num_results=num_results,
        num_burnin_steps=num_burnin_steps,
        current_state=initial_state,
        kernel=kernel
    )

    return samples

# Sample input data
filtered_temperatures_all = np.array([13.5, 14.0, 14.5, 15.0, 15.5, 16.0, 16.5, 17.0, 17.5, 18.0, 18.5, 19.0, 19.5, 20.0, 20.5, 21.0, 21.5, 22.0, 22.5, 23.0, 23.5, 24.0, 24.5])
filtered_y_values_all = np.array([
    [498.68, 462.58, 419.23, 381.86, 307.71, 235.56, 204.38, 195.61, 177.11, 160.2, 154.49, 144.5, 144.0, 145.21, 135.43, 136.55, 129.44, 127.04, 123.41, 121.04, 122.94, 118.08, 112.38],
    [200.22, 189.85, 188.68, 166.39, 153.57, 141.7, 130.1, 123.73, 121.0, 117.83, 112.3, 111.04, 108.71, 107.39, 104.66, 103.77, 101.74, 99.23, 99.02, 97.38, 97.15, 94.27, 92.51]
])

# Loop through the datasets
for j in range(len(filtered_y_values_all)):
    filtered_temperatures = filtered_temperatures_all
    filtered_y_values = filtered_y_values_all[j]

    # Perform MCMC fitting
    samples = perform_biexponential_fit(filtered_temperatures, filtered_y_values)

    # Extract the posterior samples for the parameters
    a1_samples, b1_samples, c1_samples, a2_samples, b2_samples, c2_samples = samples

    # Compute the mean of the samples to get the fitted parameters
    a1_mean = tf.reduce_mean(a1_samples).numpy()
    b1_mean = tf.reduce_mean(b1_samples).numpy()
    c1_mean = tf.reduce_mean(c1_samples).numpy()
    a2_mean = tf.reduce_mean(a2_samples).numpy()
    b2_mean = tf.reduce_mean(b2_samples).numpy()
    c2_mean = tf.reduce_mean(c2_samples).numpy()

    # Print fitted parameters for debugging
    popt = [a1_mean, b1_mean, c1_mean, a2_mean, b2_mean, c2_mean]
    print("Fitted Parameters (popt):", popt)

    # Construct equation string
    biexp_equation = f'Biexponential Fit: y = {a1_mean:.2f} * exp(-{b1_mean:.2f} * x) + {c1_mean:.2f} + {a2_mean:.2f} * exp(-{b2_mean:.2f} * x) + {c2_mean:.2f}'
    print(biexp_equation)  # This replaces equation_box += biexp_equation for demonstration

    # Evaluate the fitted biexponential function at the continuous x-values
    temperatures = np.linspace(np.min(temperatures), np.max(temperatures), 100)
    y_values_biexp_interp = biexponential(temperatures, a1_mean, b1_mean, c1_mean, a2_mean, b2_mean, c2_mean).numpy()

    # Store the interpolated y-values for this x-value
    y_values_biexp_interp_all.append(y_values_biexp_interp)
print(y_values_biexp_interp_all)



# Perform biexponential fit using filtered data
for j in range(len(x_values)):
    filtered_temperatures = np.array(filtered_temperatures_all)
    filtered_y_values = filtered_y_values_all[j]
    try:
        # Initial guesses for parameters
        initial_guesses = [max(filtered_y_values), 0.5, min(filtered_y_values), max(filtered_y_values), 0.5, min(filtered_y_values)]

        popt, _ = curve_fit(biexponential, np.array(filtered_temperatures), np.array(filtered_y_values))
        print("Fitted Parameters (popt):", popt)  # Print fitted parameters for debugging

        # Construct equation string
        biexp_equation = f'Biexponential Fit: {x_values[j]} nm: y = {popt[0]:.2f} * exp(-{popt[1]:.2f} * x) + {popt[2]:.2f} + {popt[3]:.2f} * exp(-{popt[4]:.2f} * x) + {popt[5]:.2f}'
        equation_box += biexp_equation

        # Evaluate the fitted biexponential function at the continuous x-values
        y_values_biexp_interp = biexponential(np.array(temperatures), *popt)

        # Store the interpolated y-values for this x-value
        y_values_biexp_interp_all.append(y_values_biexp_interp)
    except RuntimeError as e:
        print(f"Curve fitting failed for spectra within the specified range: {e}")

## Plot data series and fits


# Find temperatures of the first and last spectrum
first_temp = 10 + first_spectrum * 0.5
last_temp = 10 + last_spectrum * 0.5

# Define more line styles for interpolation curves
line_styles = ['-', '--', '-.', ':', '-', '--', '-.', ':', '-', '--', '-.', ':', '-', '--']




# Find maximum y-value in the scatter plot data
max_y_value = max(max(y) for y in y_values_all)

# Calculate 10% above the maximum y-value
y_upper_limit = max_y_value * 1.1  # 10% above the maximum value


# Plot interpolation curve for each x-value
plt.figure(figsize=(40, 24))
equation_box = ""
for i, x in enumerate(x_values):
# Choose line style for polynomial fit
    poly_line_style = line_styles[i]  # Cycle through line styles
    # Choose line style for biexponential fit
    biexp_line_style = line_styles[len(x_values) + i]  # Cycle through line styles after the polynomial fits
      
# Plot interpolation curve for the current x-value
    plt.plot(temperatures, y_values_interp_all[i], label=f'Polynomial Fitted Function (x={x})', linewidth=2, linestyle=poly_line_style)
# Plot biexp interpolation curve for the current x-value
    plt.plot(temperatures, y_values_biexp_interp_all[i], label=f'Biexponentieal fitted Function (x={x})', linewidth=2, linestyle=biexp_line_style)
# Plot data points
    plt.scatter(temperatures, y_values_all[i], label=f'Data Points (x={x})', color='red')

## Display equations in a box

# Define the position of the box
box_x = 10
box_y = 300
plt.text(box_x, box_y, equation_box, fontsize=20, verticalalignment='top', bbox=dict(facecolor='white', alpha=0.5))

 
plt.xlabel('Temperature [°C]', fontsize=20)
plt.ylabel('Intensity [a.u.]', fontsize=20)
plt.title('Polynomial Interpolation (5th order) of Y-Values at Fixed X-Values', fontsize=30)
plt.legend(fontsize=20)
# Adjust font size of grid numbers on axes
# Use temperatures for x-axis ticks
plt.xticks(temperatures, fontsize=20)
plt.yticks(fontsize=20)

# Set y-axis limits
plt.ylim(0, y_upper_limit)

plt.grid(True)
plt.show()


Average y-values: [[423.0416, 431.4777, 451.096, 481.7503, 506.37090000000006, 525.9091000000001, 495.9603, 498.67639999999994, 462.57519999999994, 419.23369999999994, 381.8616, 307.7102, 235.55839999999998, 204.37900000000002, 195.6081, 177.1061, 160.2024, 154.4918, 144.49539999999996, 143.99970000000002, 145.2141, 135.4279, 136.55490000000003, 129.44490000000002, 127.0415, 123.40859999999998, 121.04100000000001, 122.9365, 118.0837, 112.383, 111.109], [224.82930000000002, 212.3168, 214.2502, 215.24500000000003, 219.11010000000002, 218.8272, 202.3894, 200.21890000000002, 189.85380000000004, 188.6805, 166.3858, 153.57430000000005, 141.6951, 130.10240000000002, 123.7301, 121.002, 117.828, 112.29920000000001, 111.0444, 108.7105, 107.3886, 104.6617, 103.7737, 101.74324, 99.22576, 99.01657, 97.38140999999999, 97.15183000000002, 94.26642000000001, 92.50715, 91.93906000000001]]
[ 0.          0.6122449   1.2244898   1.83673469  2.44897959  3.06122449
  3.67346939  4.28571429  4.89795918  5.510

2024-05-18 19:18:52.585405: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
/home/jwm/.local/lib/python3.10/site-packages/tensorflow_probability/python/mcmc/sample.py:339: UserWarning: Tracing all kernel results by default is deprecated. Set the `trace_fn` argument to None (the future default value) or an explicit callback that traces the values you are interested in.
  warnings.warn('Tracing all kernel results by default is deprecated. Set '
